<a href="https://colab.research.google.com/github/GabrielBatavia/Learn_TensorFlow/blob/main/NLP_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sqeuences text or speech).

Another common term for NLP problems is sequence to sequence problems

## Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-9d2e3168-adeb-75a7-4d2f-5e62e3d7e912)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-05-11 08:20:48--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-05-11 08:20:48 (63.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as diaster or not diaster)



In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-05-11 08:20:53--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.183.207, 142.251.180.207, 142.251.172.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.183.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2024-05-11 08:20:53 (112 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualize a text dataset

In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
train_df_shuffled["text"][2227]

'The f$&amp;@ing things I do for #GISHWHES Just got soaked in a deluge going for pads and tampons. Thx @mishacollins @/@'

In [8]:
# What does the test dataframe look like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples of each class
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [10]:
# How many total sample
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)# create random indexes
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"Target: {target}", "(real diaster)" if target > 0 else "(not real diaster)")
  print(f"Text:\n{text}\n")
  print("==-\n")


Target: 0 (not real diaster)
Text:
SHOUOUT TO @kasad1lla CAUSE HER VOCALS ARE BLAZING HOT LIKE THE WEATHER SHES IN

==-

Target: 1 (real diaster)
Text:
driving the avalanche after having my car for a week is like driving a tank

==-

Target: 0 (not real diaster)
Text:
@aptly_engineerd There is no such curfew.

==-

Target: 1 (real diaster)
Text:
Three Israeli soldiers wounded in West Bank terrorist attack via /r/worldnews http://t.co/9TyucdWh3g

==-

Target: 0 (not real diaster)
Text:
Reddit's new content policy goes into effect many horrible subreddits banned or quarantined http://t.co/4oNvxncz8w http://t.co/tnggXNm6k8

==-



### Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# Use train_test_split to split training data into training and validation sets

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data to validation
                                                                            random_state=42)

In [14]:
# check len data
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [15]:
# check some example
train_sentences[:5], train_labels[:5]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
       dtype=object),
 array([0, 0, 1, 0, 0]))

## Convert text into numbers

Tokenization vs embeddings

When dealing with a text problem, one of the first things we cant do before we build a model is convers our text to numbers.

There are a few ways to do this, namely:
* Tokenziation - direct mapping of token (a token could be a word or a character) to number
* Embedding = create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [17]:
 # Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=10000, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words
                                    output_mode="int",
                                    output_sequence_length = None, # if its None its automaticly use the max long length
                                    pad_to_max_tokens = True)

In [18]:
train_sentences[0].split(), len(train_sentences[0].split())

(['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet'], 7)

In [19]:
# Find the average number of tokens in the training tweets
round(sum([len(i.split()) for i in train_sentences])) / len(train_sentences)

14.901036345059115

In [20]:
# Setup text vectorization variables
max_vocab_length = 10000  # max number of words to have in our vocabulary
max_length = 15  # max length our sequences will be (r.g. how many words from a Tweet foes a model see)

# recreate and spesific the value
text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_sequence_length = max_length)

In [21]:
# Fit the text vectorizer to training text
text_vectorizer.adapt(train_sentences)

In [22]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a blood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 353,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [23]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text : \n {random_sentence} \n Vectorized version :")
text_vectorizer([random_sentence])


Original text : 
 @Tim_A_Roberts w/o giving up too much of Nana France reminds me of America right before the war in Iraq. A restlessness leading to disaster 
 Vectorized version :


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[7561, 2225, 1287,   27,  150,  187,    6,    1, 1021, 2005,   31,
           6,  826,  179,  239]])>

In [24]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all unique words
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Number 5 most common words :\n {top_5_words}")
print(f"Number 5 least common words :\n {bottom_5_words}")

Number of words in vocab: 10000
Number 5 most common words :
 ['', '[UNK]', 'the', 'a', 'in']
Number 5 least common words :
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding layer

To make our embedding, we're going to use tensorflow embedding layers

The parameters we care most about for our embedding layer:
* input_dim = the size of our vocabulary
* output_dim = the size of the output embedding vector, for example : a value of 100 would mean each token gets represented by a vector 100 long
* input_length = length of  the sequences being passed to the embedding layers

In [25]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length, # set input shape
                             output_dim = 128, # output shape
                             input_length = max_length) # how long is each input

embedding

In [26]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text : \n {random_sentence} \n Embedded version :")

# Embed the random sentence (turn it into dense of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text : 
 @ThatSabineGirl you get me sis. this planet could do with a huge dose of obliteration. 
 Embedded version :


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00633197, -0.02248989, -0.01467304, ...,  0.01975665,
          0.00982162, -0.0299105 ],
        [-0.00610664,  0.04964694, -0.03263   , ..., -0.02361765,
         -0.01838877, -0.04252554],
        [ 0.03421605, -0.02737564, -0.01718725, ..., -0.04251533,
         -0.02282559, -0.03574941],
        ...,
        [ 0.04740741, -0.03365058, -0.02050141, ..., -0.00097357,
         -0.00325525,  0.0108787 ],
        [ 0.00152598,  0.01609753, -0.03786222, ...,  0.00877503,
         -0.00556072,  0.02297482],
        [ 0.04874801, -0.03183933, -0.01866734, ...,  0.03794297,
         -0.02444981,  0.02207201]]], dtype=float32)>

In [27]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00633197, -0.02248989, -0.01467304,  0.00257883, -0.03069375,
         0.01257843,  0.03086964,  0.03181655, -0.03697473,  0.03419319,
         0.04038019,  0.0107616 , -0.04521359,  0.00625896,  0.04795051,
        -0.01309824, -0.04500812, -0.01107324,  0.01273603,  0.01999393,
        -0.02149762,  0.00175731,  0.01892424, -0.02027321, -0.04448792,
        -0.02372961,  0.02942635, -0.04140957,  0.00449768, -0.0304783 ,
         0.02356895, -0.00582998, -0.00730865, -0.033406  , -0.02595027,
         0.04714542,  0.02504246,  0.04325496,  0.00586574, -0.00284685,
        -0.04075041,  0.02144886,  0.00570024, -0.03233199, -0.04258679,
         0.04051076,  0.04215399,  0.04498317, -0.04307627, -0.04267636,
         0.01557807, -0.00403699, -0.0117516 , -0.03187692,  0.03127055,
        -0.01044276, -0.0343039 ,  0.02614773,  0.04001081,  0.02985031,
         0.01739133, -0.04021246, -0.01932496, -0.00736736,  0.02458271,
  

## Modelling a text dataset (running a series of experiments)

we'll start with a baseline and move on from there

* Model 0 : Naive bayes (baseline)
* Model 1 : Feed_forward neural netwrok (dense model)
* Model 2 : LSTM model (RNN)
* Model 3 : GPU model (RNN)
* Model 4 : Bidirectional-LSTM model (RNN)
* Model 5 : 1D Convolutional Neural Network (CNN)
* Model 6 : TensorFlow Hub Pretrained Feature Extractor (using transder learning for NLP)
* Model 7 : Same as model 6 with 10% of training data


### Model 0 : Getting a baseline

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modeling pipeline

model_0 = Pipeline(
    [
        ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
        ("clf", MultinomialNB()) # model the text
    ]
)

# Fit the pipleine to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
# Evaluate our model 0
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score * 100:2f}%")

Our baseline model achieves an accuracy of: 79.265092%


In [30]:
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [31]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])